In [ ]:
import pandas as pd

# Мердж таблиц

Таблица со всей статистикой игроков.

In [ ]:
# player_stats = pd.read_csv('/content/drive/MyDrive/xg_project/Players data.csv') #Старая таблица
player_stats = pd.read_csv('/content/drive/MyDrive/xg_project/Players data 2.csv')
player_stats['Unnamed: 0'] = player_stats.index #в старой таблице был такой столбец, некоторые функию его используют
player_stats.replace(',','', regex=True, inplace=True) # В столбце с минутами тысячи отделены запятыми, удаляем их и кастим к флоату
player_stats['minutes'] = player_stats['minutes'].astype('float')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (59) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
player_stats

Функция пытающаяся восстановить сезон. Вызывается отдельно для каждого игрока.

In [ ]:
def guess_year(df):
  max_age = df.iloc[-1]['age']
  res = df.copy()
  early_end = 0
  for i in range(1,4):
    for index, row in df[df['age'] == max_age - 4 + i].iterrows():
      if (row['comp_level'] in [ '1. Premier League', '1. Bundesliga', '1. Serie A	', '1. La Liga', '1. Ligue 1']) and (row['xg'] != row['xg']):
        early_end = i
        break
    if early_end != i:
      break
  res['year'] = res['age'] + 2020 - max_age - early_end
  return res

Функция возвращающая список таблиц с трансферами для каждого игрока


In [ ]:
def get_seasons(player, table, max_seasons, min_games):
  
  df = table[table['name'] == player]
  df = guess_year(df)
  is_keeper = (df.iloc[-1,:]['all_goals_against_gk'] == df.iloc[-1,:]['all_goals_against_gk'])
  if (is_keeper):
    # df = df[df['psxg_gk'] == df['psxg_gk']]
    return ([], is_keeper)
  else:
    df = df[df['xg'] == df['xg']]

  df = df[df['comp_level'] != '1. MLS'] #убираем млс

  #убрали сезоны с неполной статистикой
  if (len(df['squad'].unique()) < 2):
    print("Ошибка: Только одна команда со статистикой для " + player)
    return ([], is_keeper)
  
  # В начало таблицы могло попасть начало сезона в другой команде, по которой
  # мало статистики, например Валенсия у Канселу, избавимся от этого
  if (df.iloc[0,:]['age'] == df.iloc[1,:]['age']):
    df = df.iloc[1:,:]

  # составим список всех трансферов
  cur_team = df.iloc[0,:]['squad']
  transfers = []
  for i in range(1, len(df.index)):
    new_team = df.iloc[i,:]['squad']
    if (cur_team != new_team):
      if (df.iloc[i,:]['Unnamed: 0'] - 1 == df.iloc[i - 1,:]['Unnamed: 0'] and df.iloc[i,:]['games'] >= min_games):
        transfers.append(i)
      cur_team = new_team

  season_tables = [] #список с датафреймами для каждого трансфера игрока
  for i in transfers:
    old_team = df.iloc[i - 1,:]['squad']
    old_age = df.iloc[i - 1,:]['age']
    new_team = df.iloc[i,:]['squad']
    new_age = df.iloc[i,:]['age']
    l = i - 1
    while (l > 0):
      if (old_team != df.iloc[l - 1,:]['squad'] or old_age - 1 != df.iloc[l - 1,:]['age']):
        break
      l -= 1
    r = i
    while (r < len(df.index) - 1):
      if (new_team != df.iloc[r + 1,:]['squad'] or new_age + 1 != df.iloc[r + 1,:]['age']):
        break
      r += 1
    #обрежем, если взяли больше сезонов чем нужно
    l = max(l, i - max_seasons)
    r = min(r, i + max_seasons - 1)
    season_tables.append(df.iloc[l:r + 1,:])
  return (season_tables, is_keeper)

Строим списки датафреймов для каждого игрока, ограничиваем 2 годами до и после трансфера.

In [ ]:
players = list(player_stats.name.unique())
field = []
keepers = []
for player in players:
    transfers, is_keeper = get_seasons(player, player_stats, 1, 3)
    if is_keeper:
        for df in transfers:
            keepers.append(df)
    else:
        for df in transfers:
            field.append(df)

Импортируем статистику клубов

In [ ]:
team_stats1 = pd.read_csv('/content/drive/MyDrive/xg_project/team_stats1.csv')
team_stats2 = pd.read_csv('/content/drive/MyDrive/xg_project/team_stats2.csv')
team_stats3 = pd.read_csv('/content/drive/MyDrive/xg_project/team_stats3.csv')
team_stats4 = pd.read_csv('/content/drive/MyDrive/xg_project/team_stats4.csv')
team_stats = pd.concat([team_stats1, team_stats2, team_stats3, team_stats4])

In [ ]:
team_stats

In [ ]:
field[701]

Функция для получения названий команд до и после трансфера, года последнего сезона в старой команде и окна трансфера. Летний трансфер может считатьс за зимний, если игрок успел провести несколько матчей за старую команду до трансфера.

In [ ]:
def get_info(table):
  #возвращает (название 1 команды, название 2 команды, год начала статистики, последний год в первой команде, окно трансфера (1 если зимний))
  y = table.iloc[0]['year']
  y0 = y
  team1 = table.iloc[0]['squad']
  i = 0;
  while (table.iloc[i]['squad'] == team1):
    y = table.iloc[i]['year']
    i += 1
  team2 = table.iloc[i]['squad']
  winter = (y == table.iloc[i]['year'])
  return (team1, team2, y0, y, winter)

In [ ]:
print(get_info(field[701]))

('Atalanta', 'Milan', 2019.0, 2019.0, True)


Функция для получения нужных таблиц с командной статистикой

In [ ]:
def get_tables(table):
  t1, t2, y0, y, is_win = get_info(table)
  df1 = team_stats[team_stats['team'] == t1]
  df1 = df1[(df1['year'] >= y0) & (df1['year'] <= y)]
  df2 = team_stats[team_stats['team'] == t2]
  df2 = df2[(df2['year'] >= y0) & (df2['year'] <= y)]
  return [df1, df2]

In [ ]:
merge_table.append(merge)

,team_old,year_old,result_old,goals_for_old,goals_against_old,xg_for_old,xg_against_old,possession_old,shots_on_target_against_old,goals_against_gk_old,saves_old,save_pct_old,clean_sheets_old,psxg_gk_old,psxg_net_gk_old,pens_att_gk_old,pens_allowed_old,pens_saved_old,pens_missed_gk_old,passes_completed_launched_gk_old,passes_launched_gk_old,passes_pct_launched_gk_old,passes_gk_old,passes_throws_gk_old,pct_passes_launched_gk_old,passes_length_avg_gk_old,goal_kicks_old,pct_goal_kicks_launched_old,goal_kick_length_avg_old,crosses_gk_old,crosses_stopped_gk_old,crosses_stopped_pct_gk_old,def_actions_outside_pen_area_gk_old,avg_distance_def_actions_gk_old,goals_old,shots_total_old,shots_on_target_old,shots_on_target_pct_old,goals_per_shot_old,goals_per_shot_on_target_old,...,wins_gk_y,draws_gk_y,losses_gk_y,clean_sheets_y,clean_sheets_pct_y,pens_att_gk_y,pens_allowed_y,pens_saved_y,pens_missed_gk_y,free_kick_goals_against_gk_y,corner_kick_goals_against_gk_y,own_goals_against_gk_y,psxg_gk_y,psnpxg_per_shot_on_target_against_y,psxg_net_gk_y,psxg_net_per90_gk_y,passes_completed_launched_gk_y,passes_launched_gk_y,passes_pct_launched_gk_y,passes_gk_y,passes_throws_gk_y,pct_passes_launched_gk_y,passes_length_avg_gk_y,goal_kicks_y,pct_goal_kicks_launched_y,goal_kick_length_avg_y,crosses_gk_y,crosses_stopped_gk_y,crosses_stopped_pct_gk_y,def_actions_outside_pen_area_gk_y,def_actions_outside_pen_area_per90_gk_y,avg_distance_def_actions_gk_y,dom_lg_goals_against_gk_y,dom_lg_clean_sheets_y,dom_cup_goals_against_gk_y,dom_cup_clean_sheets_y,intl_cup_goals_against_gk_y,intl_cup_clean_sheets_y,all_goals_against_gk_y,all_clean_sheets_y
0,Montpellier,2017,1.342105,0.947368,0.868421,1.115789,1.105263,46.894737,3.184211,0.868421,2.447368,74.318919,0.368421,1.002632,0.134211,0.157895,0.131579,0.000000,0.026316,5.710526,13.421053,43.176316,20.026316,4.684211,53.294737,42.042105,4.236842,63.159459,48.632432,8.842105,1.210526,14.173684,0.473684,16.368421,0.947368,13.184211,4.184211,33.873684,0.074474,0.212895,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Montpellier,2018,1.552632,1.394737,1.105263,1.189474,1.150000,48.131579,3.605263,1.105263,2.657895,65.986842,0.315789,1.089474,0.063158,0.131579,0.078947,0.026316,0.026316,9.052632,20.236842,46.628947,26.842105,4.473684,56.976316,44.450000,7.315789,66.963158,51.123684,8.210526,0.578947,8.536842,0.684211,15.576316,1.315789,13.131579,4.421053,34.352632,0.099211,0.278158,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Функция для корректировки стат на время

In [ ]:
def time_adj(df, list_of_attr):
  for attr in list_of_attr:
    df[attr] = (df[attr] * 90) / df['minutes']

Функция для корректировки стат на владение (считается что уже скорректировано на время) 

Если игрок провел в команде только половину сезона, то владение все равно берется по всему сезону

In [ ]:
def poss_adj(df, list_of_attr, on_ball): 
  # list_of_attr - список столбцов которые нужно скорректировать
  # on_ball - True для показателей на мяче (корректировка вниз при владении > 50%) и False без мяча (корр вверх)
  for i, row in df.iterrows():
    team = row['squad']
    year = row['year']
    dft = team_stats[team_stats['team'] == team]
    dft = dft[dft['year'] == year]
    dft = dft.iloc[0]
    if (dft['possession'] != dft['possession']):
      raise NotImplementedError #пока не решил что делать с комаднами без владения
    if on_ball:
      for attr in list_of_attr:
        df.loc[i, attr] = (row[attr] * 50) / dft['possession']
    else:
      for attr in list_of_attr:
        df.loc[i, attr] = (row[attr] * 50) / (100 - dft['possession'])

# Код Акима

In [ ]:
from tqdm.notebook import tqdm

merge_table = pd.DataFrame()
for observation in tqdm(field):
    old, new = get_tables(observation)
    old.columns = [x + '_old' for x in old.columns]
    new.columns = [x + '_new' for x in new.columns]

    old.reset_index(drop=True, inplace=True)
    new.reset_index(drop=True, inplace=True)

    club_features = pd.concat([old, new], axis=1)
    # club_features = pd.DataFrame(club_features.mean()).T
    club_features['year'] = club_features['year_old']
    club_features['predict_year'] = club_features['year_old']
    club_features

    observation['predict_year'] = observation['year'] - 1

    merge_before_target = club_features.merge(observation.drop(columns='predict_year'), on=['year'], how='left')
    merge = merge_before_target.merge(observation.drop(columns='year'), on=['predict_year'], how='left')

    merge_table = merge_table.append(merge)


merge_table.drop(columns = ['year_old', 'year_new', 'predict_year'], inplace=True)

merge_table.head()

,team_old,result_old,goals_for_old,goals_against_old,xg_for_old,xg_against_old,possession_old,shots_on_target_against_old,goals_against_gk_old,saves_old,save_pct_old,clean_sheets_old,psxg_gk_old,psxg_net_gk_old,pens_att_gk_old,pens_allowed_old,pens_saved_old,pens_missed_gk_old,passes_completed_launched_gk_old,passes_launched_gk_old,passes_pct_launched_gk_old,passes_gk_old,passes_throws_gk_old,pct_passes_launched_gk_old,passes_length_avg_gk_old,goal_kicks_old,pct_goal_kicks_launched_old,goal_kick_length_avg_old,crosses_gk_old,crosses_stopped_gk_old,crosses_stopped_pct_gk_old,def_actions_outside_pen_area_gk_old,avg_distance_def_actions_gk_old,goals_old,shots_total_old,shots_on_target_old,shots_on_target_pct_old,goals_per_shot_old,goals_per_shot_on_target_old,average_shot_distance_old,...,touches_def_3rd_y,touches_mid_3rd_y,touches_att_3rd_y,touches_att_pen_area_y,touches_live_ball_y,dribbles_completed_y,dribbles_y,dribbles_completed_pct_y,players_dribbled_past_y,nutmegs_y,carries_y,carry_distance_y,carry_progressive_distance_y,progressive_carries_y,carries_into_final_third_y,carries_into_penalty_area_y,miscontrols_y,dispossessed_y,pass_targets_y,passes_received_y,passes_received_pct_y,progressive_passes_received_y,on_xg_for_y,on_xg_against_y,xg_plus_minus_y,xg_plus_minus_per90_y,xg_plus_minus_wowy_y,ball_recoveries_y,aerials_won_y,aerials_lost_y,aerials_won_pct_y,dom_lg_goals_against_gk_y,dom_lg_clean_sheets_y,dom_cup_goals_against_gk_y,dom_cup_clean_sheets_y,intl_cup_goals_against_gk_y,intl_cup_clean_sheets_y,all_goals_against_gk_y,all_clean_sheets_y,Unnamed: 0_y
0,Girona,0.973684,0.973684,1.394737,1.015789,1.328947,48.026316,5.210526,1.394737,4.078947,74.723684,0.236842,1.602632,0.260526,0.210526,0.210526,0.000000,0.000000,6.342105,16.368421,41.139474,22.947368,4.921053,46.715789,40.765789,8.421053,64.613158,51.150000,10.394737,0.789474,6.828947,0.394737,13.024324,0.973684,10.368421,3.684211,35.413158,0.081842,0.222500,17.407895,...,155.0,221.0,108.0,4.0,368.0,10.0,20.0,50.0,12.0,1.0,262.0,1331.0,828.0,42.0,15.0,2.0,5.0,12.0,269.0,238.0,88.5,18.0,6.1,14.0,-7.8,-0.95,-0.85,64.0,5.0,9.0,35.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0
0,Levante,1.210526,1.157895,1.526316,1.110526,1.592105,42.789474,4.684211,1.526316,3.368421,70.362162,0.263158,1.613158,0.165789,0.157895,0.131579,0.000000,0.026316,6.526316,16.315789,40.350000,15.500000,1.921053,66.371053,49.168421,5.973684,90.528947,62.542105,10.842105,1.157895,8.894737,0.657895,14.894737,1.131579,11.105263,3.157895,30.192105,0.106842,0.307027,17.936842,...,1.0,7.0,14.0,0.0,15.0,0.0,0.0,NaN,0.0,0.0,13.0,99.0,77.0,3.0,3.0,0.0,2.0,1.0,15.0,13.0,86.7,2.0,0.2,0.4,-0.1,-0.49,-0.17,2.0,0.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.0
0,Napoli,1.940000,1.800000,0.960000,1.774000,0.962000,58.720000,3.600000,0.960000,2.780000,75.356250,0.360000,0.978000,0.058000,0.100000,0.100000,0.000000,0.000000,3.640000,10.820000,36.218367,21.820000,4.020000,31.824000,32.970000,7.160000,50.040000,43.598000,8.020000,0.700000,9.018000,0.320000,13.385417,1.760000,17.560000,5.880000,34.112000,0.101000,0.286400,18.244000,...,1368.0,969.0,51.0,21.0,2209.0,3.0,3.0,100.0,3.0,0.0,1325.0,7803.0,4454.0,71.0,5.0,1.0,1.0,1.0,1509.0,1492.0,98.9,11.0,55.3,40.9,14.4,0.40,1.36,338.0,78.0,41.0,65.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.0
0,Köln,0.700000,1.050000,1.950000,1.244118,2.000000,47.875000,5.625000,1.950000,3.875000,70.137500,0.150000,2.100000,0.041176,0.225000,0.200000,0.025000,0.000000,8.617647,22.205882,37.164706,22.764706,2.411765,67.044118,49.261765,8.205882,86.376471,61.352941,10.882353,0.823529,8.735294,0.970588,15.794118,1.000000,11.925000,3.750000,31.462500,0.081250,0.293590,17.964706,...,30.0,168.0,185.0,44.0,357.0,8.0,17.0,47.1,10.0,1.0,244.0,1092.0,426.0,27.0,6.0,2.0,15.0,16.0,437.0,280.0,64.1,92.0,13.3,14.6,-1.4,-0.18,-0.07,36.0,27.0,27.0,50.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,68.0
0,Fiorentina,1.500000,1.421053,1.210526,1.510526,1.118421,52.578947,4.000000,1.210526,2.763158,75.078947,0.368421,1.226316,0.015789

In [ ]:
from sklearn.model_selection import train_test_split
import lightgbm as lgb

In [ ]:
club_features   = ['possession', 'npxg', 'xg_against', 'passes_pct', 'touches']
player_features = ['minutes_90s', 'goals_per90', 'assists_per90', 'sca_per90', 'gca_per90']

In [ ]:
columns = [x + '_old' for x in club_features] + [x + '_new' for x in club_features] + [x + '_x' for x in player_features] + ['sca_per90_y']

In [ ]:
[x for x in old.columns if 'xg' in x]

['xg_for_old',
 'xg_against_old',
 'psxg_gk_old',
 'psxg_net_gk_old',
 'xg_old',
 'npxg_old',
 'npxg_per_shot_old',
 'xg_net_old',
 'npxg_net_old']

In [ ]:
df = merge_table.loc[:, columns].dropna(subset=['sca_per90_y'])

In [ ]:
train, valid_plus_test = train_test_split(df, random_state=2021, test_size=0.3)
val, test = train_test_split(valid_plus_test, random_state=2021, test_size=0.67)

In [ ]:
val

,possession_old,npxg_old,xg_against_old,passes_pct_old,touches_old,possession_new,npxg_new,xg_against_new,passes_pct_new,touches_new,minutes_90s_x,goals_per90_x,assists_per90_x,sca_per90_x,gca_per90_x,sca_per90_y
0,53.289474,1.105263,1.105263,82.386842,663.000000,54.535714,1.117857,1.492857,85.692857,674.178571,0.0,0.00,0.00,0.00,0.00,3.63
0,55.105263,1.076316,1.392105,80.765789,667.921053,50.578947,0.939474,1.268421,79.678947,583.026316,16.9,0.00,0.06,1.90,0.24,1.79
0,49.605263,1.118421,1.386842,75.773684,572.842105,42.894737,1.102632,1.407895,75.157895,531.131579,19.7,0.15,0.10,2.59,0.41,4.12
0,49.921053,1.100000,1.255263,74.755263,583.763158,53.062500,1.556250,1.164583,78.527083,616.833333,35.0,0.06,0.00,0.77,0.09,0.49
0,60.347826,1.506522,1.219565,85.602174,751.804348,57.528302,1.466038,1.298113,81.739623,693.528302,13.9,0.22,0.00,3.61,0.14,2.61
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,45.605263,1.144737,1.465789,73.105263,532.184211,50.250000,1.193750,1.295833,78.650000,584.770833,23.7,0.00,0.04,1.39,0.21,1.35
0,53.475000,1.787500,1.222500,80.387500,655.625000,38.558824,1.120588,1.611765,69.700000,481.000000,6.8,0.00,0.00,0.88,0.15,1.37
0,45.714286,0.821429,1.271429,76.471429,578.357143,NaN,NaN,NaN,NaN,NaN,22.2,0.09,0.22,2.52,0.36,3.90
0,52.645833,1.241667,1.241667,80.802083,648.541667,62.608696,2.110870,1.008696,87.304348,764.130435,15.6,0.13,0.19,2.31,0.38,2.63


In [ ]:
target_column = 'sca_per90_y'
lgb_train = lgb.Dataset(train.drop(columns=target_column).values, train[target_column].values)
lgb_val = lgb.Dataset(val.drop(columns=target_column).values, val[target_column].values)


params = {
    "objective" : "mse",
    "metric" : "mae",
    "max_depth" : 7,
    "num_leaves" : 100,
    "learning_rate" : 0.05,
    "bagging_fraction" : 0.3,
    "feature_fraction" : 0.15,
    "lambda_l1" : 5,
    "lambda_l2" : 5,
    "bagging_seed" : 42,
    "verbosity" : 1,
    "seed": 42
}

clf = lgb.train(
    params,
    lgb_train,
    valid_sets=[lgb_val],
    num_boost_round=500,
    early_stopping_rounds=10,
    verbose_eval=10
)

Training until validation scores don't improve for 10 rounds.
[10]	valid_0's l1: 0.853709
[20]	valid_0's l1: 0.811723
[30]	valid_0's l1: 0.789985
[40]	valid_0's l1: 0.757916
[50]	valid_0's l1: 0.743729
[60]	valid_0's l1: 0.736203
[70]	valid_0's l1: 0.728917
[80]	valid_0's l1: 0.714704
[90]	valid_0's l1: 0.700718
[100]	valid_0's l1: 0.694517
[110]	valid_0's l1: 0.693271
[120]	valid_0's l1: 0.688364
[130]	valid_0's l1: 0.686504
[140]	valid_0's l1: 0.682479
[150]	valid_0's l1: 0.680528
[160]	valid_0's l1: 0.677822
[170]	valid_0's l1: 0.67445
[180]	valid_0's l1: 0.672049
[190]	valid_0's l1: 0.671537
Early stopping, best iteration is:
[185]	valid_0's l1: 0.671309
